<h2>Ставим библиотеку для работы с поиском opensearch-py </h2>
<ol>
<li>Запускаем докер образ с настройками
<li>Проверяем что экземпляр поиска доступен
</ol>

<textarea style=" min-width:500px; max-width:100%;min-height:150px;height:100%;width:100%;">
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org opensearch-py

docker run -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "OPENSEARCH_INITIAL_ADMIN_PASSWORD=sea12323>RCh" opensearchproject/opensearch:latest

curl https://localhost:9200 -ku admin:sea12323>RCh
</textarea>

In [12]:
from opensearchpy import OpenSearch
import pprint as pp

host = 'localhost'
port = 9200
auth = ('admin', 'sea12323>RCh') # For testing only. Don't store credentials in code.
#ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Optional client certificates if you don't want to use HTTP basic authentication.
# client_cert_path = '/full/path/to/client.pem'
# client_key_path = '/full/path/to/client-key.pem'

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    #ca_certs = ca_certs_path
)



<p> Для настройки морфологии </p>
"russian_morphology": {
  "type":     "hunspell",
  "locale": "ru_RU",
  "dedup": "true"
},
"english_morphology": {
  "type":     "hunspell",
  "locale": "en_US",
  "dedup": "true"
},


"russian_morphology", "english_morphology", 

<h2>Синонимы без графа синонимов и со стоп словами</h2>

In [115]:
# Create an index with non-default settings.
index_name = 'python-test-index'
# Delete the index.
try:
    response = client.indices.delete(index = index_name )
except:
    print ("NOT DELETE INDEX")

print('\nDeleting index:')
print(response)

index_body = {
  "settings": {
    "analysis": {
      "filter": {
        
            "keywords_fio" : {
              "keywords" : [
             "путин", "собянин"
              ],
              "type" : "keyword_marker"
            },
            "keywords_abbr" : {
              "keywords" : [
            "гвс", "мвд", "гибдд", "гаи"
              ],
              "type" : "keyword_marker"
            },
            "synonyms_abbr" : {
              "type" : "synonym_graph",
              "synonyms" : [
"драть как сидорову козу, сильно бить, пороть",
"дышать ладан, болезненный вид человек, болезненный человек, астеник",
"тридевять земель, далеко",
"затрапезный вид, неряшливость одежде",
"как гуся вода, безразлично, переносить трудности",
"конь валялся, работа началась, ничего сделано, ничего сделано",
"кровь молоком,  здоровый человек",
"несолоно хлебавши, достиг результата, безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно",
              ]
            },
            "synonyms_abbr_query" : {
              "type" : "synonym",
              "synonyms" : [
"пуха пера, суеверие",
"бриллиантовая рука, виноватая пришел, рук, люди булочную такси ездят",
"иван васильевич меняет профессию, удачно зашел, лепота, танцуют",
"кавказская пленница,  короче склифасовский, свадьбы будет, украл верну, туфля спасибо",
"москва слезам верит, хэллоу общежитие слушает, хуже ихних, засосало мещанское болото",
"служебный роман, значит хорошие сапоги брать, женщине должна загадка",
"горе ума, счастливые часов наблюдают, свежо предание верится трудом, служить рад прислуживаться тошно, судьи, герой моего романа, ба знакомые лица",
"евгений онегин, можно дельным человеком думать красе ногтей,  меньше женщину любим легче нравимся"
              ]
            },
            "synonyms_fio" : {
              "type" : "synonym",
              "synonyms" : [
                "николая, николаю, николая, николаем, николае, николай",
                "киры, кире, киру, кирой, кире, кира",
              ]
            },
            "stopwords_ru" : {
              "ignore_case" : "true",
              "type" : "stop",
              "stopwords" : [
                "а", "без", "более", "бы", "был", "была", "были", "было", "быть", "в", "вам", "вас", "весь", "во", "вот", "все", "всего", "всех", "вы", "где", "да",  "даже", "для", "до", "его", "ее", "если", "есть", "еще", "же", "за", "здесь", "и", "из", "или", "им", "их", "к", "ко", "когда", "кто", "ли", "либо", "мне", "может", "мы", "на", "надо", "наш", "не", "него", "нее", "нет", "ни", "них", "но", "ну", "о", "об", "однако", "он", "она", "они", "оно", "от", "очень", "по", "под", "при", "с", "со", "так", "также", "такой", "там", "те", "тем", "то", "того", "тоже", "той", "только", "том", "ты", "у", "уже", "хотя", "чего", "чей", "чем", "что", "чтобы", "чье", "чья", "эта", "эти", "это", "я"
              ]
            }
      },
      "analyzer": {
        "default": {
          "tokenizer":  "standard", 
          "filter": ["lowercase", "stopwords_ru", "keywords_fio", "keywords_abbr", "synonyms_abbr"],
              "char_filter" : [
                "html_strip",
                "date_filter",
                "hyphen_delete_filter",
		"lowercase_split",
		"num_split_left",
		"num_split_right",
                "ru"
              ]
        },
            "default_search" : {
              "tokenizer" : "standard",
              "filter" : [ "lowercase", "stopwords_ru", "keywords_fio", "keywords_abbr", "synonyms_abbr_query"],
              "char_filter" : [
                "html_strip",
                "date_filter",
                "hyphen_delete_filter",
		"lowercase_split",
		"num_split_left",
		"num_split_right",
                "ru"
              ]
            }
      },
          "char_filter" : {
            "date_filter" : {
              "pattern" : "(\\d+)[-/.](?=\\d+)",
              "type" : "pattern_replace",
              "replacement" : "$1_"
            },
            "lowercase_split" : {
              "pattern" : "([а-яa-z]+)([А-ЯA-Z]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "num_split_left" : {
              "pattern" : "([а-яa-zА-ЯA-Z]+)([0-9]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "num_split_right" : {
              "pattern" : "([0-9]+)([а-яa-zА-ЯA-Z]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "ru" : {
              "type" : "mapping",
              "mappings" : [
                "Ё=>Е",
                "ё=>е"
              ]
            },
            "hyphen_delete_filter" : {  
              "pattern" : "(\\s*)-(\\s+)", 
              "type" : "pattern_replace",
              "replacement" : "$1"
            }
          }
    }
  }
}
response = client.indices.create(index_name, body=index_body)
print('\nCreating index:')
print(response)



Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}


In [116]:
# Add a document to the index.
document = {
'title': 'Драть как сидорову козу',
'text': 'Драть как сидорову козу (прост.) – сильно, жестоко и безжалостно пороть, бить кого-либо. По наиболее распространенной версии, выражение связано с тем, что именем Сидор в народе называли зажиточного, но скупого и мелочного человеке. Далее объяснения оборота разнятся: 1) Сидор и свою козу даже за мелкую провинность драл (бил) безжалостно; 2) в сравнении отраженно стремление мести Сидору: если он сам недосягаем для наказаний, то пусть хоть его козе достанется основательно.'

}

i = 1

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Астеник',
'text': 'Асте́ник (от греч. asthenikos — слабосильный, болезненный) дышать на ладан, болезненный на вид человек, болезненный человек — человек обладающий типом строения тела, для которого характерны высокий рост, худощавость, длинная шея, продольно вытянутый череп, узкое резко очерченное лицо[1]. Немецкий учёный Эрнст Кречмер опубликовал в 1921 году работу, в которой связывал астенический тип с определёнными чертами темперамента и характера: повышенной чувствительностью и одновременно холодностью, нервностью и угловатостью движений, необщительностью, склонностью к «уходу в себя». Современная наука подобную связь отрицает.'

}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)
    
i += 1
document = {
'title': 'за тридевять земель',
'text': 'очень далеко ◆ За тридевять земель в тридесятом царстве, за огненной рекою живёт баба-яга. «Марья Моревна» ◆ Вот читаю лекцию, на вид я весел, а самому так и хочется крикнуть во всё горло или полететь куда-нибудь за тридевять земель. А. П. Чехов, «О вреде табака», 1886 г.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'затрапезный вид',
'text': 'Несколько устаревший фразеологизм заставляет вспомнить слово «трапеза», что по-гречески означает «стол», отсюда «затрапезный» — «застольный». Сегодня мы употребляем это слово в значении «захудалый», «низкосортный». Но разве не принято к обеденному столу одеваться красиво? Оказывается, так было не везде и не всегда. Когда-то в духовных училищах и семинариях дети бедного провинциального духовенства ходили в старых, потертых халатах. Именно за трапезами можно было увидеть множество таких одежд, получивших название «затрапезных». Правда, по другой версии лингвистов, у выражения есть еще одно объяснение, отсылающее к петровским временам. Среди предпринимателей, получивших от Петра фабрики «в поощрение способностей и полезной деятельности», был Иван Затрапезнов, развернувший производство на ярославской текстильной мануфактуре, которая быстро приняла его красочную фамилию в качестве «бренда».'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Почему говорят как с гуся вода?',
'text': '«Как с гуся вода» – говорят про человека, которому повезло в какой-нибудь опасной ситуации. А ещё про такого счастливчика можно сказать, что он «вышел сухим из воды». Представь, гуси (и другие водоплавающие птицы) большую часть дня проводят в пруду и при этом ни капельки не промокают! Их перья покрыты особым слоем жира, который отталкивает воду. Поэтому гуси выходят из воды совершенно сухими! А вот сказать «как с вороны вода» – нельзя. Тот, кто видел мокрую ворону, знает, что она выглядит очень жалкой и несчастной. Существует, кстати, поговорка про белую ворону. Но это уже совсем другая история... '
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'ничего еще не сделано',
'text': 'Этимология конь не валялся Существует несколько версий объяснения выражения: от крестьянского обычая дать коню поваляться перед тем, как его будут запрягать или седлать; от того, что лошади катаются по земле, чтобы на них не надевали седло или хомут; от обрядового купания коня в росах; от валяния валенка с кона (мыска) или валяния войлока с кона (начала); со временем забытое слово кон превратилось в конь.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Кровь с молоком',
'text': 'Кровь с молоком (одобр.) – 1) о человеке со здоровым, свежим цветом лица, с румянцем; 2) о чьем-либо свежем, румяном лице. Выражение из русского фольклора, где соединились народные представления о красоте цвета: красного как кровь и белого как молоко. На Руси издавна признаком красоты считались белое лицо и румянец на щеках, что было свидетельством хорошего здоровья.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'БЕЗУСПЕШНО',
'text': 'Словарь синонимов Безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно, тщетно, вотще, зря, втуне, попусту, без последствий. Ни к чему это не повело, из этого ничего не вышло. Говорить на ветер. К чему вы это говорите? Только заряды на воздух тратите, никто вас не слышит. Тург. Кто-то там заботится об моей нравственности... Пусть, да ведь это все попусту! Ведь все стрельба холостыми. Дост. Уйти с пустыми руками, неудовлетворенным, не солоно хлебавши, ни с чем; остаться ни при чем, остаться (отойти) с носом. Напрасный труд; глас вопиющего в пустыне. Я надрываюсь, убеждая его, а все не впрок. Ср. Напрасно.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Ни пуха, ни пера!',
'text': 'Охотники-любители отправляются в лес в начале сезона охоты. Они делятся друг с другом своими вымышленными и реальными подвигами, развлекаются и наслаждаются девственной природой. Неожиданно отдых героев омрачается необходимостью вступить в опасную схватку с браконьерами. Фильм «Ни пуха, ни пера!» повествует о красоте лесов и озёр, общем человеческом долге любить и хранить природу, её первозданность. Охотничья комедия Виктора Иванова показывает напряжённую и, в конечном счёте, успешную охоту настоящих охотников за браконьерами, полную невероятных приключений и неожиданных поворотов действия. Как и в каждой хорошей комедии, здесь тоже присутствуют песни и любовная история.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'бриллиантовая рука',
'text': '«Бриллиа́нтовая рука́» — советская эксцентрическая комедия, снятая в 1968 году режиссёром Леонидом Гайдаем. Сценарий написан Морисом Слободским, Яковом Костюковским и Леонидом Гайдаем. В основе картины — история о том, как зарубежные контрабандисты ошибочно приняли советского туриста Семёна Семёновича Горбункова за агента-перевозчика Геннадия Козодоева[⇨]. Сюжет раскручивается вокруг настойчивых, но безуспешных попыток злоумышленников вернуть себе бриллианты, спрятанные в гипсовой повязке[⇨]. Образ главного героя создавался сценаристами с учётом актёрских данных Юрия Никулина, который попал в съёмочную группу без кинопроб. В фильме также участвовали Андрей Миронов, Анатолий Папанов, Нонна Мордюкова, Светлана Светличная, Станислав Чекан, Владимир Гуляев, Нина Гребешкова и другие артисты[⇨]. Съёмки проходили с апреля по ноябрь 1968 года в Сочи, Адлере, Туапсе, Гжатске, Павловской Слободе, на натурных объектах Москвы и в павильонах «Мосфильма». «Зарубежные сцены» снимались в Баку[⇨]. При работе над отдельными сценами создатели фильма использовали комбинированные съёмки, механические спецэффекты и другие технологии[⇨].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Иван Васильевич меняет профессию',
'text': '«Ива́н Васи́льевич меня́ет профе́ссию» — советская фантастическая комедия 1973 года, снятая режиссёром Леонидом Гайдаем по мотивам пьесы Михаила Булгакова «Иван Васильевич». Фильм рассказывает об инженере-изобретателе Шурике, создавшем машину времени, которая открывает двери в XVI век — во времена Ивана Васильевича Грозного, в результате чего царь случайно оказывается в советской Москве, а его тёзка — управдом Иван Васильевич Бунша, вместе с вором-рецидивистом Жоржем Милославским — в палатах царя.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Кавказская пленница, или Новые приключения Шурика',
'text': '«Кавка́зская пле́нница, или Но́вые приключе́ния Шу́рика» — советский художественный фильм, снятый Леонидом Гайдаем. Сценарий к картине написан Яковом Костюковским и Морисом Слободским при участии самого режиссёра[⇨]. Съёмки проходили в 1966 году в павильонах «Мосфильма», а также в Крыму — на базе Ялтинской киностудии, на Кавказе — в районе Красной Поляны и в Абхазии — в долине озера Рица[⇨]. Главные роли исполняли Александр Демьяненко, Наталья Варлей, Владимир Этуш, Фрунзик Мкртчян, Руслан Ахметов, Юрий Никулин, Георгий Вицин, Евгений Моргунов и другие[⇨]. Действие фильма происходит на Кавказе. В основе сюжета — приключения собирателя фольклора Шурика, который, находясь в южном городе, оказывается втянутым в историю с похищением девушки Нины[⇨]. Инициатор похищения — номенклатурный работник районного масштаба товарищ Саахов; в роли его кунаков выступают персонажи комедийной троицы — Трус, Балбес и Бывалый[⇨].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Москва слезам не верит',
'text': 'Москва, 1958 год. В столичном общежитии живут три 20-летние подруги, приехавшие из провинции. Их судьбы складываются именно так, как предполагает характер каждой из девушек. Целеустремлённая и сдержанная Катерина, не добрав баллов при поступлении в институт, работает штамповщицей, а позже — слесарем-наладчиком на машиностроительном заводе. Стеснительная и простая Антонина трудится маляром на стройке и замуж выходит за москвича-рабочего. Самоуверенной и весёлой Людмиле Москва представляется лотереей, в которой она должна выиграть своё особенное счастье. Она работает на хлебозаводе, но мечтает выйти замуж за статусного москвича — артиста, управленца, спортсмена, учёного. Она знакомится с Сергеем Гуриным — восходящей хоккейной звездой. Катерина, которая пошла на поводу у Людмилы и вместе с ней выдала себя за профессорскую дочь, горько расплачивается за свою авантюру. Она полюбила красавца-телеоператора Рудольфа, забеременела от него, а он, узнав, где она на самом деле работает и что она обманула его, бросает её.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Служебный роман',
'text': 'Анатолий Ефремович мечтает о повышении — освободилось место начальника отдела лёгкой промышленности. И вот в один прекрасный день в статистическое учреждение заместителем директора назначают недавно вернувшегося из-за границы Юрия Григорьевича Самохвалова (Олег Басилашвили) — давнего институтского приятеля Новосельцева и бывшего возлюбленного Ольги Рыжовой. Самохвалов достаточно обеспечен, удачно женат, живёт с шиком. Ему, разумеется, понадобятся «свои люди» на новом месте, поэтому Самохвалов пытается намекнуть Людмиле Прокофьевне на Новосельцева как на приемлемый вариант для назначения на вакантное место, но эта кандидатура не находит поддержки. Тогда Юрий Григорьевич советует своему другу приударить за Калугиной вечером у него дома, где планируется отпраздновать назначение. Новосельцев, который смертельно боится Людмилы Прокофьевны, всё равно соглашается с предложением Самохвалова. Уединившись с ней, Анатолий Ефремович пытается угостить Калугину коктейлем, завести разговор о грибах, но Людмилу Прокофьевну ни он, ни предметы разговора не интересуют. Новосельцев уходит, но потом возвращается в несколько подвыпившем состоянии и безуспешно пытается развлечь Людмилу Прокофьевну стихами, песнями и танцами. Попытки угодить надоедают Людмиле Прокофьевне, и она выгоняет Новосельцева, а тот в ответ высказывает своему директору в присутствии коллег всё, что о ней думает. После этого Людмила Прокофьевна уезжает домой.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Горе от ума',
'text': 'Замысел комедии возник в 1820 году (по некоторым данным, уже в 1816[2]), но активную работу над текстом автор начинает в Тифлисе по возвращении из Персии. Большое влияние на Грибоедова произвели рассказы его подруги детства Евгении Борисовны Греховой. К началу 1822 года написаны первые два акта, а весной и летом 1823 года в Москве завершается первый вариант пьесы. Именно здесь писатель мог пополнить наблюдения за бытом и нравами московского дворянства, «надышаться воздухом» светских гостиных. Но и потом работа не прекращается: в 1824 году возникает новый вариант, носивший название «Горе и нет ума» (первоначально — «Горе уму»).'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Евгений Онегин',
'text': '«Евге́ний Оне́гин» (дореф. «Евгеній Онѣгинъ») — роман в стихах русского поэта Александра Сергеевича Пушкина, начат 9 мая 1823 года и закончен 5 октября 1831 года, одно из самых значительных произведений русской словесности. Повествование ведётся от имени безымянного автора, который, впрочем, в первых же строфах называет Онегина «добрый мой приятель». По известному определению В. Г. Белинского, Пушкин назвал «Евгения Онегина» романом в стихах, поскольку в нём изображена «жизнь во всей её прозаической действительности»[1].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)



Adding document:
{'_index': 'python-test-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '2', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '3', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '4', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '5', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successfu

In [123]:
q = 'сильно бить'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)




q = 'астеник'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'очень далеко'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'неряшливость в одежде'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'все безразлично'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'ничего не сделано'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'здоровый человек'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'бесцельно'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'суеверие'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'только без рук'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'лепота'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'свадьбы не будет'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'а мы не хуже ихних'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'в женщине должна быть загадка'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'ба знакомые все лица'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'чем меньше женщину мы любим тем легче нравимся мы ей'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


Search results:  сильно бить
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'python-test-index',
                    '_score': 9.399224,
                    '_source': {'text': 'Драть как сидорову козу (прост.) – '
                                        'сильно, жестоко и безжалостно пороть, '
                                        'бить кого-либо. По наиболее '
                                        'распространенной версии, выражение '
                                        'связано с тем, что именем Сидор в '
                                        'народе называли зажиточного, но '
                                        'скупого и мелочного человеке. Далее '
                                        'объяснения оборота разнятся: 1) Сидор '
                                        'и свою козу даже за мелкую '
                                        'провинность драл (бил) безжалостно; '
        

<h2>Синонимы без графа синонимов и без стоп слов</h2>

In [126]:
# Create an index with non-default settings.
index_name = 'python-test-index'
# Delete the index.
try:
    response = client.indices.delete(index = index_name )
except:
    print ("NOT DELETE INDEX")

print('\nDeleting index:')
print(response)

index_body = {
  "settings": {
    "analysis": {
      "filter": {
        
            "keywords_fio" : {
              "keywords" : [
             "путин", "собянин"
              ],
              "type" : "keyword_marker"
            },
            "keywords_abbr" : {
              "keywords" : [
            "гвс", "мвд", "гибдд", "гаи"
              ],
              "type" : "keyword_marker"
            },
            "synonyms_abbr" : {
              "type" : "synonym_graph",
              "synonyms" : [
"драть как сидорову козу, сильно бить, пороть",
"дышать на ладан, болезненный на вид человек, болезненный человек, астеник",
"за тридевять земель, очень далеко",
"затрапезный вид, неряшливость в одежде",
"как с гуся вода, все безразлично, переносить трудности",
"конь не валялся, работа еще не началась, ничего еще не сделано, ничего не сделано",
"кровь с молоком,  здоровый человек",
"несолоно хлебавши, не достиг результата, безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно",
              ]
            },
            "synonyms_abbr_query" : {
              "type" : "synonym",
              "synonyms" : [
"ни пуха ни пера, суеверие",
"бриллиантовая рука, не виноватая я он сам пришел, только без рук, наши люди в булочную на такси не ездят",
"иван васильевич меняет профессию, это я удачно зашел, лепота, танцуют все",
"кавказская пленница,  короче склифасовский, свадьбы не будет, я ее украл я ее и верну, чей туфля о мое спасибо",
"москва слезам не верит, хэллоу общежитие слушает, а мы не хуже ихних, всё засосало мещанское болото",
"служебный роман, значит хорошие сапоги надо брать, в женщине должна быть загадка",
"горе от ума, счастливые часов не наблюдают, свежо предание а верится с трудом, служить бы рад прислуживаться тошно, а судьи кто, герой не моего романа, ба знакомые все лица",
"евгений онегин, быть можно дельным человеком и думать о красе ногтей,  чем меньше женщину мы любим тем легче нравимся мы ей"
              ]
            },
            "synonyms_fio" : {
              "type" : "synonym",
              "synonyms" : [
                "николая, николаю, николая, николаем, николае, николай",
                "киры, кире, киру, кирой, кире, кира",
              ]
            },
            "stopwords_ru" : {
              "ignore_case" : "true",
              "type" : "stop",
              "stopwords" : [
                "аолфыварывфралрывал"
              ]
            }
      },
      "analyzer": {
        "default": {
          "tokenizer":  "standard", 
          "filter": ["lowercase", "stopwords_ru", "keywords_fio", "keywords_abbr", "synonyms_abbr"],
              "char_filter" : [
                "html_strip",
                "date_filter",
                "hyphen_delete_filter",
		"lowercase_split",
		"num_split_left",
		"num_split_right",
                "ru"
              ]
        },
            "default_search" : {
              "tokenizer" : "standard",
              "filter" : [ "lowercase", "stopwords_ru", "keywords_fio", "keywords_abbr", "synonyms_abbr_query"],
              "char_filter" : [
                "html_strip",
                "date_filter",
                "hyphen_delete_filter",
		"lowercase_split",
		"num_split_left",
		"num_split_right",
                "ru"
              ]
            }
      },
          "char_filter" : {
            "date_filter" : {
              "pattern" : "(\\d+)[-/.](?=\\d+)",
              "type" : "pattern_replace",
              "replacement" : "$1_"
            },
            "lowercase_split" : {
              "pattern" : "([а-яa-z]+)([А-ЯA-Z]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "num_split_left" : {
              "pattern" : "([а-яa-zА-ЯA-Z]+)([0-9]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "num_split_right" : {
              "pattern" : "([0-9]+)([а-яa-zА-ЯA-Z]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "ru" : {
              "type" : "mapping",
              "mappings" : [
                "Ё=>Е",
                "ё=>е"
              ]
            },
            "hyphen_delete_filter" : {  
              "pattern" : "(\\s*)-(\\s+)", 
              "type" : "pattern_replace",
              "replacement" : "$1"
            }
          }
    }
  }
}
response = client.indices.create(index_name, body=index_body)
print('\nCreating index:')
print(response)



Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}


In [127]:
# Add a document to the index.
document = {
'title': 'Драть как сидорову козу',
'text': 'Драть как сидорову козу (прост.) – сильно, жестоко и безжалостно пороть, бить кого-либо. По наиболее распространенной версии, выражение связано с тем, что именем Сидор в народе называли зажиточного, но скупого и мелочного человеке. Далее объяснения оборота разнятся: 1) Сидор и свою козу даже за мелкую провинность драл (бил) безжалостно; 2) в сравнении отраженно стремление мести Сидору: если он сам недосягаем для наказаний, то пусть хоть его козе достанется основательно.'

}

i = 1

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Астеник',
'text': 'Асте́ник (от греч. asthenikos — слабосильный, болезненный) дышать на ладан, болезненный на вид человек, болезненный человек — человек обладающий типом строения тела, для которого характерны высокий рост, худощавость, длинная шея, продольно вытянутый череп, узкое резко очерченное лицо[1]. Немецкий учёный Эрнст Кречмер опубликовал в 1921 году работу, в которой связывал астенический тип с определёнными чертами темперамента и характера: повышенной чувствительностью и одновременно холодностью, нервностью и угловатостью движений, необщительностью, склонностью к «уходу в себя». Современная наука подобную связь отрицает.'

}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)
    
i += 1
document = {
'title': 'за тридевять земель',
'text': 'очень далеко ◆ За тридевять земель в тридесятом царстве, за огненной рекою живёт баба-яга. «Марья Моревна» ◆ Вот читаю лекцию, на вид я весел, а самому так и хочется крикнуть во всё горло или полететь куда-нибудь за тридевять земель. А. П. Чехов, «О вреде табака», 1886 г.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'затрапезный вид',
'text': 'Несколько устаревший фразеологизм заставляет вспомнить слово «трапеза», что по-гречески означает «стол», отсюда «затрапезный» — «застольный». Сегодня мы употребляем это слово в значении «захудалый», «низкосортный». Но разве не принято к обеденному столу одеваться красиво? Оказывается, так было не везде и не всегда. Когда-то в духовных училищах и семинариях дети бедного провинциального духовенства ходили в старых, потертых халатах. Именно за трапезами можно было увидеть множество таких одежд, получивших название «затрапезных». Правда, по другой версии лингвистов, у выражения есть еще одно объяснение, отсылающее к петровским временам. Среди предпринимателей, получивших от Петра фабрики «в поощрение способностей и полезной деятельности», был Иван Затрапезнов, развернувший производство на ярославской текстильной мануфактуре, которая быстро приняла его красочную фамилию в качестве «бренда».'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Почему говорят как с гуся вода?',
'text': '«Как с гуся вода» – говорят про человека, которому повезло в какой-нибудь опасной ситуации. А ещё про такого счастливчика можно сказать, что он «вышел сухим из воды». Представь, гуси (и другие водоплавающие птицы) большую часть дня проводят в пруду и при этом ни капельки не промокают! Их перья покрыты особым слоем жира, который отталкивает воду. Поэтому гуси выходят из воды совершенно сухими! А вот сказать «как с вороны вода» – нельзя. Тот, кто видел мокрую ворону, знает, что она выглядит очень жалкой и несчастной. Существует, кстати, поговорка про белую ворону. Но это уже совсем другая история... '
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'ничего еще не сделано',
'text': 'Этимология конь не валялся Существует несколько версий объяснения выражения: от крестьянского обычая дать коню поваляться перед тем, как его будут запрягать или седлать; от того, что лошади катаются по земле, чтобы на них не надевали седло или хомут; от обрядового купания коня в росах; от валяния валенка с кона (мыска) или валяния войлока с кона (начала); со временем забытое слово кон превратилось в конь.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Кровь с молоком',
'text': 'Кровь с молоком (одобр.) – 1) о человеке со здоровым, свежим цветом лица, с румянцем; 2) о чьем-либо свежем, румяном лице. Выражение из русского фольклора, где соединились народные представления о красоте цвета: красного как кровь и белого как молоко. На Руси издавна признаком красоты считались белое лицо и румянец на щеках, что было свидетельством хорошего здоровья.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'БЕЗУСПЕШНО',
'text': 'Словарь синонимов Безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно, тщетно, вотще, зря, втуне, попусту, без последствий. Ни к чему это не повело, из этого ничего не вышло. Говорить на ветер. К чему вы это говорите? Только заряды на воздух тратите, никто вас не слышит. Тург. Кто-то там заботится об моей нравственности... Пусть, да ведь это все попусту! Ведь все стрельба холостыми. Дост. Уйти с пустыми руками, неудовлетворенным, не солоно хлебавши, ни с чем; остаться ни при чем, остаться (отойти) с носом. Напрасный труд; глас вопиющего в пустыне. Я надрываюсь, убеждая его, а все не впрок. Ср. Напрасно.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Ни пуха, ни пера!',
'text': 'Охотники-любители отправляются в лес в начале сезона охоты. Они делятся друг с другом своими вымышленными и реальными подвигами, развлекаются и наслаждаются девственной природой. Неожиданно отдых героев омрачается необходимостью вступить в опасную схватку с браконьерами. Фильм «Ни пуха, ни пера!» повествует о красоте лесов и озёр, общем человеческом долге любить и хранить природу, её первозданность. Охотничья комедия Виктора Иванова показывает напряжённую и, в конечном счёте, успешную охоту настоящих охотников за браконьерами, полную невероятных приключений и неожиданных поворотов действия. Как и в каждой хорошей комедии, здесь тоже присутствуют песни и любовная история.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'бриллиантовая рука',
'text': '«Бриллиа́нтовая рука́» — советская эксцентрическая комедия, снятая в 1968 году режиссёром Леонидом Гайдаем. Сценарий написан Морисом Слободским, Яковом Костюковским и Леонидом Гайдаем. В основе картины — история о том, как зарубежные контрабандисты ошибочно приняли советского туриста Семёна Семёновича Горбункова за агента-перевозчика Геннадия Козодоева[⇨]. Сюжет раскручивается вокруг настойчивых, но безуспешных попыток злоумышленников вернуть себе бриллианты, спрятанные в гипсовой повязке[⇨]. Образ главного героя создавался сценаристами с учётом актёрских данных Юрия Никулина, который попал в съёмочную группу без кинопроб. В фильме также участвовали Андрей Миронов, Анатолий Папанов, Нонна Мордюкова, Светлана Светличная, Станислав Чекан, Владимир Гуляев, Нина Гребешкова и другие артисты[⇨]. Съёмки проходили с апреля по ноябрь 1968 года в Сочи, Адлере, Туапсе, Гжатске, Павловской Слободе, на натурных объектах Москвы и в павильонах «Мосфильма». «Зарубежные сцены» снимались в Баку[⇨]. При работе над отдельными сценами создатели фильма использовали комбинированные съёмки, механические спецэффекты и другие технологии[⇨].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Иван Васильевич меняет профессию',
'text': '«Ива́н Васи́льевич меня́ет профе́ссию» — советская фантастическая комедия 1973 года, снятая режиссёром Леонидом Гайдаем по мотивам пьесы Михаила Булгакова «Иван Васильевич». Фильм рассказывает об инженере-изобретателе Шурике, создавшем машину времени, которая открывает двери в XVI век — во времена Ивана Васильевича Грозного, в результате чего царь случайно оказывается в советской Москве, а его тёзка — управдом Иван Васильевич Бунша, вместе с вором-рецидивистом Жоржем Милославским — в палатах царя.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Кавказская пленница, или Новые приключения Шурика',
'text': '«Кавка́зская пле́нница, или Но́вые приключе́ния Шу́рика» — советский художественный фильм, снятый Леонидом Гайдаем. Сценарий к картине написан Яковом Костюковским и Морисом Слободским при участии самого режиссёра[⇨]. Съёмки проходили в 1966 году в павильонах «Мосфильма», а также в Крыму — на базе Ялтинской киностудии, на Кавказе — в районе Красной Поляны и в Абхазии — в долине озера Рица[⇨]. Главные роли исполняли Александр Демьяненко, Наталья Варлей, Владимир Этуш, Фрунзик Мкртчян, Руслан Ахметов, Юрий Никулин, Георгий Вицин, Евгений Моргунов и другие[⇨]. Действие фильма происходит на Кавказе. В основе сюжета — приключения собирателя фольклора Шурика, который, находясь в южном городе, оказывается втянутым в историю с похищением девушки Нины[⇨]. Инициатор похищения — номенклатурный работник районного масштаба товарищ Саахов; в роли его кунаков выступают персонажи комедийной троицы — Трус, Балбес и Бывалый[⇨].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Москва слезам не верит',
'text': 'Москва, 1958 год. В столичном общежитии живут три 20-летние подруги, приехавшие из провинции. Их судьбы складываются именно так, как предполагает характер каждой из девушек. Целеустремлённая и сдержанная Катерина, не добрав баллов при поступлении в институт, работает штамповщицей, а позже — слесарем-наладчиком на машиностроительном заводе. Стеснительная и простая Антонина трудится маляром на стройке и замуж выходит за москвича-рабочего. Самоуверенной и весёлой Людмиле Москва представляется лотереей, в которой она должна выиграть своё особенное счастье. Она работает на хлебозаводе, но мечтает выйти замуж за статусного москвича — артиста, управленца, спортсмена, учёного. Она знакомится с Сергеем Гуриным — восходящей хоккейной звездой. Катерина, которая пошла на поводу у Людмилы и вместе с ней выдала себя за профессорскую дочь, горько расплачивается за свою авантюру. Она полюбила красавца-телеоператора Рудольфа, забеременела от него, а он, узнав, где она на самом деле работает и что она обманула его, бросает её.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Служебный роман',
'text': 'Анатолий Ефремович мечтает о повышении — освободилось место начальника отдела лёгкой промышленности. И вот в один прекрасный день в статистическое учреждение заместителем директора назначают недавно вернувшегося из-за границы Юрия Григорьевича Самохвалова (Олег Басилашвили) — давнего институтского приятеля Новосельцева и бывшего возлюбленного Ольги Рыжовой. Самохвалов достаточно обеспечен, удачно женат, живёт с шиком. Ему, разумеется, понадобятся «свои люди» на новом месте, поэтому Самохвалов пытается намекнуть Людмиле Прокофьевне на Новосельцева как на приемлемый вариант для назначения на вакантное место, но эта кандидатура не находит поддержки. Тогда Юрий Григорьевич советует своему другу приударить за Калугиной вечером у него дома, где планируется отпраздновать назначение. Новосельцев, который смертельно боится Людмилы Прокофьевны, всё равно соглашается с предложением Самохвалова. Уединившись с ней, Анатолий Ефремович пытается угостить Калугину коктейлем, завести разговор о грибах, но Людмилу Прокофьевну ни он, ни предметы разговора не интересуют. Новосельцев уходит, но потом возвращается в несколько подвыпившем состоянии и безуспешно пытается развлечь Людмилу Прокофьевну стихами, песнями и танцами. Попытки угодить надоедают Людмиле Прокофьевне, и она выгоняет Новосельцева, а тот в ответ высказывает своему директору в присутствии коллег всё, что о ней думает. После этого Людмила Прокофьевна уезжает домой.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Горе от ума',
'text': 'Замысел комедии возник в 1820 году (по некоторым данным, уже в 1816[2]), но активную работу над текстом автор начинает в Тифлисе по возвращении из Персии. Большое влияние на Грибоедова произвели рассказы его подруги детства Евгении Борисовны Греховой. К началу 1822 года написаны первые два акта, а весной и летом 1823 года в Москве завершается первый вариант пьесы. Именно здесь писатель мог пополнить наблюдения за бытом и нравами московского дворянства, «надышаться воздухом» светских гостиных. Но и потом работа не прекращается: в 1824 году возникает новый вариант, носивший название «Горе и нет ума» (первоначально — «Горе уму»).'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Евгений Онегин',
'text': '«Евге́ний Оне́гин» (дореф. «Евгеній Онѣгинъ») — роман в стихах русского поэта Александра Сергеевича Пушкина, начат 9 мая 1823 года и закончен 5 октября 1831 года, одно из самых значительных произведений русской словесности. Повествование ведётся от имени безымянного автора, который, впрочем, в первых же строфах называет Онегина «добрый мой приятель». По известному определению В. Г. Белинского, Пушкин назвал «Евгения Онегина» романом в стихах, поскольку в нём изображена «жизнь во всей её прозаической действительности»[1].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)



Adding document:
{'_index': 'python-test-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '2', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '3', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '4', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '5', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successfu

In [128]:
q = 'сильно бить'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)




q = 'астеник'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'очень далеко'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'неряшливость в одежде'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'все безразлично'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'ничего не сделано'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'здоровый человек'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'бесцельно'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'суеверие'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'только без рук'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'лепота'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'свадьбы не будет'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'а мы не хуже ихних'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'в женщине должна быть загадка'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'ба знакомые все лица'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'чем меньше женщину мы любим тем легче нравимся мы ей'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


Search results:  сильно бить
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'python-test-index',
                    '_score': 10.341058,
                    '_source': {'text': 'Драть как сидорову козу (прост.) – '
                                        'сильно, жестоко и безжалостно пороть, '
                                        'бить кого-либо. По наиболее '
                                        'распространенной версии, выражение '
                                        'связано с тем, что именем Сидор в '
                                        'народе называли зажиточного, но '
                                        'скупого и мелочного человеке. Далее '
                                        'объяснения оборота разнятся: 1) Сидор '
                                        'и свою козу даже за мелкую '
                                        'провинность драл (бил) безжалостно; '
       

<h2>Синонимы с графом синонимов и без стоп слов</h2>

In [130]:
# Create an index with non-default settings.
index_name = 'python-test-index'
# Delete the index.
try:
    response = client.indices.delete(index = index_name )
except:
    print ("NOT DELETE INDEX")

print('\nDeleting index:')
print(response)

index_body = {
  "settings": {
    "analysis": {
      "filter": {
        
            "keywords_fio" : {
              "keywords" : [
             "путин", "собянин"
              ],
              "type" : "keyword_marker"
            },
            "keywords_abbr" : {
              "keywords" : [
            "гвс", "мвд", "гибдд", "гаи"
              ],
              "type" : "keyword_marker"
            },
            "synonyms_abbr" : {
              "type" : "synonym_graph",
              "synonyms" : [
"коза, козу, бить, драть как сидорову козу, сильно бить, пороть => коза, козу, бить, драть_как_сидорову_козу, сильно_бить, пороть",
"болезнь, недуг, дышать на ладан, болезненный на вид человек, болезненный человек, астеник => болезнь, недуг, дышать_на_ладан, болезненный_на_вид_человек, болезненный_человек, астеник",
"далеко, за тридевять земель, очень далеко => далеко, за_тридевять_земель, очень_далеко",
"бомж, затрапезный вид, неряшливость в одежде => бомж, затрапезный_вид, неряшливость_в_одежде",
"пофигист, как с гуся вода, все безразлично, переносить трудности => пофигист, как_с_гуся_вода, все_безразлично, переносить_трудности",
"лень, прокрастинация, конь не валялся, работа еще не началась, ничего еще не сделано, ничего не сделано = лень, прокрастинация, конь_не_валялся, работа_еще_не_началась, ничего_еще_не_сделано, ничего_не_сделано",
"здоровье, бычара, кровь с молоком,  здоровый человек => здоровье, бычара, кровь_с_молоком,  здоровый_человек",
"безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно, несолоно хлебавши, не достиг результата => несолоно_хлебавши, не_достиг_результата, безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно",
"удачи, ни пуха ни пера, суеверие => удачи, ни_пуха_ни_пера, суеверие",
              ]
            },
            "synonyms_abbr_query" : {
              "type" : "synonym",
              "synonyms" : [
"рука, бриллиантовая рука, не виноватая я он сам пришел, только без рук, наши люди в булочную на такси не ездят => рука, бриллиантовая_рука, не_виноватая_я_он_сам_пришел, только_без_рук, наши_люди_в_булочную_на_такси_не_ездят",
"иван, васильевич, иван васильевич меняет профессию, это я удачно зашел, лепота, танцуют все => иван, васильевич, иван_васильевич_меняет_профессию, это_я_удачно_зашел, лепота, танцуют_все",
"кавказ, пленница, кавказская пленница,  короче склифасовский, свадьбы не будет, я ее украл я ее и верну, чей туфля о мое спасибо => кавказ, пленница, кавказская_пленница,  короче_склифасовский, свадьбы_не_будет, я_ее_украл_я_ее_и_верну, чей_туфля_о_мое_спасибо",
"москва, слезы, москва слезам не верит, хэллоу общежитие слушает, а мы не хуже ихних, всё засосало мещанское болото => москва, слезы, москва_слезам_не_верит, хэллоу_общежитие_слушает, а_мы_не_хуже_ихних, всё_засосало_мещанское_болото",
"роман, сапоги, служебный роман, значит хорошие сапоги надо брать, в женщине должна быть загадка => роман, сапоги, служебный_роман, значит_хорошие_сапоги_надо_брать, в_женщине_должна_быть_загадка",
"горе, ум, горе от ума, счастливые часов не наблюдают, свежо предание а верится с трудом, служить бы рад прислуживаться тошно, а судьи кто, герой не моего романа, ба знакомые все лица => горе, ум, горе_от_ума, счастливые_часов_не_наблюдают, свежо_предание_а_верится_с_трудом, служить_бы_рад_прислуживаться_тошно, а_судьи_кто, герой_не_моего_романа, ба_знакомые_все_лица",
"женя, евгений, евгений онегин, быть можно дельным человеком и думать о красе ногтей,  чем меньше женщину мы любим тем легче нравимся мы ей => женя, евгений, евгений_онегин, быть_можно_дельным_человеком_и_думать_о_красе_ногтей,  чем_меньше_женщину_мы_любим_тем_легче_нравимся_мы_ей"
              ]
            },
            "synonyms_fio" : {
              "type" : "synonym",
              "synonyms" : [
                "николая, николаю, николая, николаем, николае, николай",
                "киры, кире, киру, кирой, кире, кира",
              ]
            },
            "stopwords_ru" : {
              "ignore_case" : "true",
              "type" : "stop",
              "stopwords" : [
                "аолфыварывфралрывал"
              ]
            }
      },
      "analyzer": {
        "default": {
          "tokenizer":  "standard", 
          "filter": ["lowercase", "stopwords_ru", "keywords_fio", "keywords_abbr", "synonyms_abbr"],
              "char_filter" : [
                "html_strip",
                "date_filter",
                "hyphen_delete_filter",
		"lowercase_split",
		"num_split_left",
		"num_split_right",
                "ru"
              ]
        },
            "default_search" : {
              "tokenizer" : "standard",
              "filter" : [ "lowercase", "stopwords_ru", "keywords_fio", "keywords_abbr", "synonyms_abbr_query"],
              "char_filter" : [
                "html_strip",
                "date_filter",
                "hyphen_delete_filter",
		"lowercase_split",
		"num_split_left",
		"num_split_right",
                "ru"
              ]
            }
      },
          "char_filter" : {
            "date_filter" : {
              "pattern" : "(\\d+)[-/.](?=\\d+)",
              "type" : "pattern_replace",
              "replacement" : "$1_"
            },
            "lowercase_split" : {
              "pattern" : "([а-яa-z]+)([А-ЯA-Z]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "num_split_left" : {
              "pattern" : "([а-яa-zА-ЯA-Z]+)([0-9]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "num_split_right" : {
              "pattern" : "([0-9]+)([а-яa-zА-ЯA-Z]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "ru" : {
              "type" : "mapping",
              "mappings" : [
                "Ё=>Е",
                "ё=>е"
              ]
            },
            "hyphen_delete_filter" : {  
              "pattern" : "(\\s*)-(\\s+)", 
              "type" : "pattern_replace",
              "replacement" : "$1"
            }
          }
    }
  }
}
response = client.indices.create(index_name, body=index_body)
print('\nCreating index:')
print(response)



Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}


In [131]:
# Add a document to the index.
document = {
'title': 'Драть как сидорову козу',
'text': 'Драть как сидорову козу (прост.) – сильно, жестоко и безжалостно пороть, бить кого-либо. По наиболее распространенной версии, выражение связано с тем, что именем Сидор в народе называли зажиточного, но скупого и мелочного человеке. Далее объяснения оборота разнятся: 1) Сидор и свою козу даже за мелкую провинность драл (бил) безжалостно; 2) в сравнении отраженно стремление мести Сидору: если он сам недосягаем для наказаний, то пусть хоть его козе достанется основательно.'

}

i = 1

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Астеник',
'text': 'Асте́ник (от греч. asthenikos — слабосильный, болезненный) дышать на ладан, болезненный на вид человек, болезненный человек — человек обладающий типом строения тела, для которого характерны высокий рост, худощавость, длинная шея, продольно вытянутый череп, узкое резко очерченное лицо[1]. Немецкий учёный Эрнст Кречмер опубликовал в 1921 году работу, в которой связывал астенический тип с определёнными чертами темперамента и характера: повышенной чувствительностью и одновременно холодностью, нервностью и угловатостью движений, необщительностью, склонностью к «уходу в себя». Современная наука подобную связь отрицает.'

}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)
    
i += 1
document = {
'title': 'за тридевять земель',
'text': 'очень далеко ◆ За тридевять земель в тридесятом царстве, за огненной рекою живёт баба-яга. «Марья Моревна» ◆ Вот читаю лекцию, на вид я весел, а самому так и хочется крикнуть во всё горло или полететь куда-нибудь за тридевять земель. А. П. Чехов, «О вреде табака», 1886 г.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'затрапезный вид',
'text': 'Несколько устаревший фразеологизм заставляет вспомнить слово «трапеза», что по-гречески означает «стол», отсюда «затрапезный» — «застольный». Сегодня мы употребляем это слово в значении «захудалый», «низкосортный». Но разве не принято к обеденному столу одеваться красиво? Оказывается, так было не везде и не всегда. Когда-то в духовных училищах и семинариях дети бедного провинциального духовенства ходили в старых, потертых халатах. Именно за трапезами можно было увидеть множество таких одежд, получивших название «затрапезных». Правда, по другой версии лингвистов, у выражения есть еще одно объяснение, отсылающее к петровским временам. Среди предпринимателей, получивших от Петра фабрики «в поощрение способностей и полезной деятельности», был Иван Затрапезнов, развернувший производство на ярославской текстильной мануфактуре, которая быстро приняла его красочную фамилию в качестве «бренда».'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Почему говорят как с гуся вода?',
'text': '«Как с гуся вода» – говорят про человека, которому повезло в какой-нибудь опасной ситуации. А ещё про такого счастливчика можно сказать, что он «вышел сухим из воды». Представь, гуси (и другие водоплавающие птицы) большую часть дня проводят в пруду и при этом ни капельки не промокают! Их перья покрыты особым слоем жира, который отталкивает воду. Поэтому гуси выходят из воды совершенно сухими! А вот сказать «как с вороны вода» – нельзя. Тот, кто видел мокрую ворону, знает, что она выглядит очень жалкой и несчастной. Существует, кстати, поговорка про белую ворону. Но это уже совсем другая история... '
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'ничего еще не сделано',
'text': 'Этимология конь не валялся Существует несколько версий объяснения выражения: от крестьянского обычая дать коню поваляться перед тем, как его будут запрягать или седлать; от того, что лошади катаются по земле, чтобы на них не надевали седло или хомут; от обрядового купания коня в росах; от валяния валенка с кона (мыска) или валяния войлока с кона (начала); со временем забытое слово кон превратилось в конь.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Кровь с молоком',
'text': 'Кровь с молоком (одобр.) – 1) о человеке со здоровым, свежим цветом лица, с румянцем; 2) о чьем-либо свежем, румяном лице. Выражение из русского фольклора, где соединились народные представления о красоте цвета: красного как кровь и белого как молоко. На Руси издавна признаком красоты считались белое лицо и румянец на щеках, что было свидетельством хорошего здоровья.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'БЕЗУСПЕШНО',
'text': 'Словарь синонимов Безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно, тщетно, вотще, зря, втуне, попусту, без последствий. Ни к чему это не повело, из этого ничего не вышло. Говорить на ветер. К чему вы это говорите? Только заряды на воздух тратите, никто вас не слышит. Тург. Кто-то там заботится об моей нравственности... Пусть, да ведь это все попусту! Ведь все стрельба холостыми. Дост. Уйти с пустыми руками, неудовлетворенным, не солоно хлебавши, ни с чем; остаться ни при чем, остаться (отойти) с носом. Напрасный труд; глас вопиющего в пустыне. Я надрываюсь, убеждая его, а все не впрок. Ср. Напрасно.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Ни пуха, ни пера!',
'text': 'Охотники-любители отправляются в лес в начале сезона охоты. Они делятся друг с другом своими вымышленными и реальными подвигами, развлекаются и наслаждаются девственной природой. Неожиданно отдых героев омрачается необходимостью вступить в опасную схватку с браконьерами. Фильм «Ни пуха, ни пера!» повествует о красоте лесов и озёр, общем человеческом долге любить и хранить природу, её первозданность. Охотничья комедия Виктора Иванова показывает напряжённую и, в конечном счёте, успешную охоту настоящих охотников за браконьерами, полную невероятных приключений и неожиданных поворотов действия. Как и в каждой хорошей комедии, здесь тоже присутствуют песни и любовная история.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'бриллиантовая рука',
'text': '«Бриллиа́нтовая рука́» — советская эксцентрическая комедия, снятая в 1968 году режиссёром Леонидом Гайдаем. Сценарий написан Морисом Слободским, Яковом Костюковским и Леонидом Гайдаем. В основе картины — история о том, как зарубежные контрабандисты ошибочно приняли советского туриста Семёна Семёновича Горбункова за агента-перевозчика Геннадия Козодоева[⇨]. Сюжет раскручивается вокруг настойчивых, но безуспешных попыток злоумышленников вернуть себе бриллианты, спрятанные в гипсовой повязке[⇨]. Образ главного героя создавался сценаристами с учётом актёрских данных Юрия Никулина, который попал в съёмочную группу без кинопроб. В фильме также участвовали Андрей Миронов, Анатолий Папанов, Нонна Мордюкова, Светлана Светличная, Станислав Чекан, Владимир Гуляев, Нина Гребешкова и другие артисты[⇨]. Съёмки проходили с апреля по ноябрь 1968 года в Сочи, Адлере, Туапсе, Гжатске, Павловской Слободе, на натурных объектах Москвы и в павильонах «Мосфильма». «Зарубежные сцены» снимались в Баку[⇨]. При работе над отдельными сценами создатели фильма использовали комбинированные съёмки, механические спецэффекты и другие технологии[⇨].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Иван Васильевич меняет профессию',
'text': '«Ива́н Васи́льевич меня́ет профе́ссию» — советская фантастическая комедия 1973 года, снятая режиссёром Леонидом Гайдаем по мотивам пьесы Михаила Булгакова «Иван Васильевич». Фильм рассказывает об инженере-изобретателе Шурике, создавшем машину времени, которая открывает двери в XVI век — во времена Ивана Васильевича Грозного, в результате чего царь случайно оказывается в советской Москве, а его тёзка — управдом Иван Васильевич Бунша, вместе с вором-рецидивистом Жоржем Милославским — в палатах царя.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Кавказская пленница, или Новые приключения Шурика',
'text': '«Кавка́зская пле́нница, или Но́вые приключе́ния Шу́рика» — советский художественный фильм, снятый Леонидом Гайдаем. Сценарий к картине написан Яковом Костюковским и Морисом Слободским при участии самого режиссёра[⇨]. Съёмки проходили в 1966 году в павильонах «Мосфильма», а также в Крыму — на базе Ялтинской киностудии, на Кавказе — в районе Красной Поляны и в Абхазии — в долине озера Рица[⇨]. Главные роли исполняли Александр Демьяненко, Наталья Варлей, Владимир Этуш, Фрунзик Мкртчян, Руслан Ахметов, Юрий Никулин, Георгий Вицин, Евгений Моргунов и другие[⇨]. Действие фильма происходит на Кавказе. В основе сюжета — приключения собирателя фольклора Шурика, который, находясь в южном городе, оказывается втянутым в историю с похищением девушки Нины[⇨]. Инициатор похищения — номенклатурный работник районного масштаба товарищ Саахов; в роли его кунаков выступают персонажи комедийной троицы — Трус, Балбес и Бывалый[⇨].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Москва слезам не верит',
'text': 'Москва, 1958 год. В столичном общежитии живут три 20-летние подруги, приехавшие из провинции. Их судьбы складываются именно так, как предполагает характер каждой из девушек. Целеустремлённая и сдержанная Катерина, не добрав баллов при поступлении в институт, работает штамповщицей, а позже — слесарем-наладчиком на машиностроительном заводе. Стеснительная и простая Антонина трудится маляром на стройке и замуж выходит за москвича-рабочего. Самоуверенной и весёлой Людмиле Москва представляется лотереей, в которой она должна выиграть своё особенное счастье. Она работает на хлебозаводе, но мечтает выйти замуж за статусного москвича — артиста, управленца, спортсмена, учёного. Она знакомится с Сергеем Гуриным — восходящей хоккейной звездой. Катерина, которая пошла на поводу у Людмилы и вместе с ней выдала себя за профессорскую дочь, горько расплачивается за свою авантюру. Она полюбила красавца-телеоператора Рудольфа, забеременела от него, а он, узнав, где она на самом деле работает и что она обманула его, бросает её.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Служебный роман',
'text': 'Анатолий Ефремович мечтает о повышении — освободилось место начальника отдела лёгкой промышленности. И вот в один прекрасный день в статистическое учреждение заместителем директора назначают недавно вернувшегося из-за границы Юрия Григорьевича Самохвалова (Олег Басилашвили) — давнего институтского приятеля Новосельцева и бывшего возлюбленного Ольги Рыжовой. Самохвалов достаточно обеспечен, удачно женат, живёт с шиком. Ему, разумеется, понадобятся «свои люди» на новом месте, поэтому Самохвалов пытается намекнуть Людмиле Прокофьевне на Новосельцева как на приемлемый вариант для назначения на вакантное место, но эта кандидатура не находит поддержки. Тогда Юрий Григорьевич советует своему другу приударить за Калугиной вечером у него дома, где планируется отпраздновать назначение. Новосельцев, который смертельно боится Людмилы Прокофьевны, всё равно соглашается с предложением Самохвалова. Уединившись с ней, Анатолий Ефремович пытается угостить Калугину коктейлем, завести разговор о грибах, но Людмилу Прокофьевну ни он, ни предметы разговора не интересуют. Новосельцев уходит, но потом возвращается в несколько подвыпившем состоянии и безуспешно пытается развлечь Людмилу Прокофьевну стихами, песнями и танцами. Попытки угодить надоедают Людмиле Прокофьевне, и она выгоняет Новосельцева, а тот в ответ высказывает своему директору в присутствии коллег всё, что о ней думает. После этого Людмила Прокофьевна уезжает домой.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Горе от ума',
'text': 'Замысел комедии возник в 1820 году (по некоторым данным, уже в 1816[2]), но активную работу над текстом автор начинает в Тифлисе по возвращении из Персии. Большое влияние на Грибоедова произвели рассказы его подруги детства Евгении Борисовны Греховой. К началу 1822 года написаны первые два акта, а весной и летом 1823 года в Москве завершается первый вариант пьесы. Именно здесь писатель мог пополнить наблюдения за бытом и нравами московского дворянства, «надышаться воздухом» светских гостиных. Но и потом работа не прекращается: в 1824 году возникает новый вариант, носивший название «Горе и нет ума» (первоначально — «Горе уму»).'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Евгений Онегин',
'text': '«Евге́ний Оне́гин» (дореф. «Евгеній Онѣгинъ») — роман в стихах русского поэта Александра Сергеевича Пушкина, начат 9 мая 1823 года и закончен 5 октября 1831 года, одно из самых значительных произведений русской словесности. Повествование ведётся от имени безымянного автора, который, впрочем, в первых же строфах называет Онегина «добрый мой приятель». По известному определению В. Г. Белинского, Пушкин назвал «Евгения Онегина» романом в стихах, поскольку в нём изображена «жизнь во всей её прозаической действительности»[1].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)



Adding document:
{'_index': 'python-test-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '2', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '3', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '4', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '5', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successfu

In [132]:
q = 'сильно бить'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)




q = 'астеник'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'очень далеко'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'неряшливость в одежде'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'все безразлично'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'ничего не сделано'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'здоровый человек'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'бесцельно'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'суеверие'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'только без рук'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'лепота'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'свадьбы не будет'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'а мы не хуже ихних'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'в женщине должна быть загадка'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'ба знакомые все лица'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'чем меньше женщину мы любим тем легче нравимся мы ей'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


Search results:  сильно бить
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'python-test-index',
                    '_score': 7.2500277,
                    '_source': {'text': 'Драть как сидорову козу (прост.) – '
                                        'сильно, жестоко и безжалостно пороть, '
                                        'бить кого-либо. По наиболее '
                                        'распространенной версии, выражение '
                                        'связано с тем, что именем Сидор в '
                                        'народе называли зажиточного, но '
                                        'скупого и мелочного человеке. Далее '
                                        'объяснения оборота разнятся: 1) Сидор '
                                        'и свою козу даже за мелкую '
                                        'провинность драл (бил) безжалостно; '
       

<h2>Синонимы с графом синонимов и c стоп словами</h2>

In [154]:
# Create an index with non-default settings.
index_name = 'python-test-index'
# Delete the index.
try:
    response = client.indices.delete(index = index_name )
except:
    print ("NOT DELETE INDEX")

print('\nDeleting index:')
print(response)

index_body = {
  "settings": {
    "analysis": {
      "filter": {
        
            "keywords_fio" : {
              "keywords" : [
             "путин", "собянин"
              ],
              "type" : "keyword_marker"
            },
            "keywords_abbr" : {
              "keywords" : [
            "гвс", "мвд", "гибдд", "гаи"
              ],
              "type" : "keyword_marker"
            },
            "synonyms_abbr" : {
              "type" : "synonym_graph",
              "synonyms" : [
"коза, козу, бить, драть как сидорову козу, сильно бить, пороть => коза, козу, бить, драть_как_сидорову_козу, сильно_бить, пороть",
"болезнь, недуг, дышать ладан, болезненный вид человек, болезненный человек, астеник => болезнь, недуг, дышать_ладан, болезненный_вид_человек, болезненный_человек, астеник",
"далеко, тридевять земель, далеко => далеко, тридевять_земель, очень_далеко",
"бомж, затрапезный вид, неряшливость одежде => бомж, затрапезный_вид, неряшливость_в_одежде",
"пофигист, гуся вода, безразлично, переносить трудности => пофигист, гуся_вода, все_безразлично, переносить_трудности",
"лень, прокрастинация, конь валялся, работа началась, ничего сделано, ничего сделано = лень, прокрастинация, конь_не_валялся, работа_еще_не_началась, ничего_еще_не_сделано, ничего_не_сделано",
"здоровье, бычара, кровь молоком,  здоровый человек => здоровье, бычара, кровь_с_молоком,  здоровый_человек",
"безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно, несолоно хлебавши, достиг результата => несолоно_хлебавши, не_достиг_результата, безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно",
"удачи, пуха пера, суеверие => удачи, ни_пуха_ни_пера, суеверие",
              ]
            },
            "synonyms_abbr_query" : {
              "type" : "synonym",
              "synonyms" : [
"рука, бриллиантовая рука, виноватая пришел, рук, люди булочную такси ездят => рука, бриллиантовая_рука, не_виноватая_я_он_сам_пришел, только_без_рук, наши_люди_в_булочную_на_такси_не_ездят",
"иван, васильевич, иван васильевич меняет профессию, удачно зашел, лепота, танцуют => иван, васильевич, иван_васильевич_меняет_профессию, это_я_удачно_зашел, лепота, танцуют_все",
"кавказ, пленница, кавказская пленница,  короче склифасовский, свадьбы будет, украл верну, туфля спасибо => кавказ, пленница, кавказская_пленница,  короче_склифасовский, свадьбы_не_будет, я_ее_украл_я_ее_и_верну, чей_туфля_о_мое_спасибо",
"москва, слезы, москва слезам верит, хэллоу общежитие слушает, хуже ихних, засосало мещанское болото => москва, слезы, москва_слезам_не_верит, хэллоу_общежитие_слушает, а_мы_не_хуже_ихних, всё_засосало_мещанское_болото",
"роман, сапоги, служебный роман, значит хорошие сапоги брать, женщине должна загадка => роман, сапоги, служебный_роман, значит_хорошие_сапоги_надо_брать, в_женщине_должна_быть_загадка",
"горе, ум, горе ума, счастливые часов наблюдают, свежо предание верится трудом, служить рад прислуживаться тошно, герой романа, ба знакомые лица => горе, ум, горе_от_ума, счастливые_часов_не_наблюдают, свежо_предание_а_верится_с_трудом, служить_бы_рад_прислуживаться_тошно, а_судьи_кто, герой_не_моего_романа, ба_знакомые_все_лица",
"женя, евгений, евгений онегин, можно дельным человеком думать красе ногтей,  меньше женщину любим легче нравимся => женя, евгений, евгений_онегин, быть_можно_дельным_человеком_и_думать_о_красе_ногтей,  чем_меньше_женщину_мы_любим_тем_легче_нравимся_мы_ей"
              ]
            },
            "synonyms_fio" : {
              "type" : "synonym",
              "synonyms" : [
                "николая, николаю, николая, николаем, николае, николай",
                "киры, кире, киру, кирой, кире, кира",
              ]
            },
            "stopwords_ru" : {
              "ignore_case" : "true",
              "type" : "stop",
              "stopwords" : [
                "а", "без", "более", "бы", "был", "была", "были", "было", "быть", "в", "вам", "вас", "весь", "во", "вот", "все", "всего", "всех", "вы", "где", "да",  "даже", "для", "до", "его", "ее", "если", "есть", "еще", "же", "за", "здесь", "и", "из", "или", "им", "их", "к", "ко", "когда", "кто", "ли", "либо", "мне", "может", "мы", "на", "надо", "наш", "не", "него", "нее", "нет", "ни", "них", "но", "ну", "о", "об", "однако", "он", "она", "они", "оно", "от", "очень", "по", "под", "при", "с", "со", "так", "также", "такой", "там", "те", "тем", "то", "того", "тоже", "той", "только", "том", "ты", "у", "уже", "хотя", "чего", "чей", "чем", "что", "чтобы", "чье", "чья", "эта", "эти", "это", "я"
              ]
            }
      },
      "analyzer": {
        "default": {
          "tokenizer":  "standard", 
          "filter": ["lowercase", "stopwords_ru", "keywords_fio", "keywords_abbr", "synonyms_abbr"],
              "char_filter" : [
                "html_strip",
                "date_filter",
                "hyphen_delete_filter",
		"lowercase_split",
		"num_split_left",
		"num_split_right",
                "ru"
              ]
        },
            "default_search" : {
              "tokenizer" : "standard",
              "filter" : [ "lowercase", "stopwords_ru", "keywords_fio", "keywords_abbr", "synonyms_abbr_query"],
              "char_filter" : [
                "html_strip",
                "date_filter",
                "hyphen_delete_filter",
		"lowercase_split",
		"num_split_left",
		"num_split_right",
                "ru"
              ]
            }
      },
          "char_filter" : {
            "date_filter" : {
              "pattern" : "(\\d+)[-/.](?=\\d+)",
              "type" : "pattern_replace",
              "replacement" : "$1_"
            },
            "lowercase_split" : {
              "pattern" : "([а-яa-z]+)([А-ЯA-Z]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "num_split_left" : {
              "pattern" : "([а-яa-zА-ЯA-Z]+)([0-9]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "num_split_right" : {
              "pattern" : "([0-9]+)([а-яa-zА-ЯA-Z]+)",
              "type" : "pattern_replace",
              "replacement" : "$1 $2"
            },
            "ru" : {
              "type" : "mapping",
              "mappings" : [
                "Ё=>Е",
                "ё=>е"
              ]
            },
            "hyphen_delete_filter" : {  
              "pattern" : "(\\s*)-(\\s+)", 
              "type" : "pattern_replace",
              "replacement" : "$1"
            }
          }
    }
  }
}
response = client.indices.create(index_name, body=index_body)
print('\nCreating index:')
print(response)



Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}


In [155]:
# Add a document to the index.
document = {
'title': 'Драть как сидорову козу',
'text': 'Драть как сидорову козу (прост.) – сильно, жестоко и безжалостно пороть, бить кого-либо. По наиболее распространенной версии, выражение связано с тем, что именем Сидор в народе называли зажиточного, но скупого и мелочного человеке. Далее объяснения оборота разнятся: 1) Сидор и свою козу даже за мелкую провинность драл (бил) безжалостно; 2) в сравнении отраженно стремление мести Сидору: если он сам недосягаем для наказаний, то пусть хоть его козе достанется основательно.'

}

i = 1

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Астеник',
'text': 'Асте́ник (от греч. asthenikos — слабосильный, болезненный) дышать на ладан, болезненный на вид человек, болезненный человек — человек обладающий типом строения тела, для которого характерны высокий рост, худощавость, длинная шея, продольно вытянутый череп, узкое резко очерченное лицо[1]. Немецкий учёный Эрнст Кречмер опубликовал в 1921 году работу, в которой связывал астенический тип с определёнными чертами темперамента и характера: повышенной чувствительностью и одновременно холодностью, нервностью и угловатостью движений, необщительностью, склонностью к «уходу в себя». Современная наука подобную связь отрицает.'

}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)
    
i += 1
document = {
'title': 'за тридевять земель',
'text': 'очень далеко ◆ За тридевять земель в тридесятом царстве, за огненной рекою живёт баба-яга. «Марья Моревна» ◆ Вот читаю лекцию, на вид я весел, а самому так и хочется крикнуть во всё горло или полететь куда-нибудь за тридевять земель. А. П. Чехов, «О вреде табака», 1886 г.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'затрапезный вид',
'text': 'Несколько устаревший фразеологизм заставляет вспомнить слово «трапеза», что по-гречески означает «стол», отсюда «затрапезный» — «застольный». Сегодня мы употребляем это слово в значении «захудалый», «низкосортный». Но разве не принято к обеденному столу одеваться красиво? Оказывается, так было не везде и не всегда. Когда-то в духовных училищах и семинариях дети бедного провинциального духовенства ходили в старых, потертых халатах. Именно за трапезами можно было увидеть множество таких одежд, получивших название «затрапезных». Правда, по другой версии лингвистов, у выражения есть еще одно объяснение, отсылающее к петровским временам. Среди предпринимателей, получивших от Петра фабрики «в поощрение способностей и полезной деятельности», был Иван Затрапезнов, развернувший производство на ярославской текстильной мануфактуре, которая быстро приняла его красочную фамилию в качестве «бренда».'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Почему говорят как с гуся вода?',
'text': '«Как с гуся вода» – говорят про человека, которому повезло в какой-нибудь опасной ситуации. А ещё про такого счастливчика можно сказать, что он «вышел сухим из воды». Представь, гуси (и другие водоплавающие птицы) большую часть дня проводят в пруду и при этом ни капельки не промокают! Их перья покрыты особым слоем жира, который отталкивает воду. Поэтому гуси выходят из воды совершенно сухими! А вот сказать «как с вороны вода» – нельзя. Тот, кто видел мокрую ворону, знает, что она выглядит очень жалкой и несчастной. Существует, кстати, поговорка про белую ворону. Но это уже совсем другая история... '
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'ничего еще не сделано',
'text': 'Этимология конь не валялся Существует несколько версий объяснения выражения: от крестьянского обычая дать коню поваляться перед тем, как его будут запрягать или седлать; от того, что лошади катаются по земле, чтобы на них не надевали седло или хомут; от обрядового купания коня в росах; от валяния валенка с кона (мыска) или валяния войлока с кона (начала); со временем забытое слово кон превратилось в конь.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Кровь с молоком',
'text': 'Кровь с молоком (одобр.) – 1) о человеке со здоровым, свежим цветом лица, с румянцем; 2) о чьем-либо свежем, румяном лице. Выражение из русского фольклора, где соединились народные представления о красоте цвета: красного как кровь и белого как молоко. На Руси издавна признаком красоты считались белое лицо и румянец на щеках, что было свидетельством хорошего здоровья.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'БЕЗУСПЕШНО',
'text': 'Словарь синонимов Безуспешно, бесплодно, бесполезно, бесцельно, напрасно, неудачно, тщетно, вотще, зря, втуне, попусту, без последствий. Ни к чему это не повело, из этого ничего не вышло. Говорить на ветер. К чему вы это говорите? Только заряды на воздух тратите, никто вас не слышит. Тург. Кто-то там заботится об моей нравственности... Пусть, да ведь это все попусту! Ведь все стрельба холостыми. Дост. Уйти с пустыми руками, неудовлетворенным, не солоно хлебавши, ни с чем; остаться ни при чем, остаться (отойти) с носом. Напрасный труд; глас вопиющего в пустыне. Я надрываюсь, убеждая его, а все не впрок. Ср. Напрасно.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Ни пуха, ни пера!',
'text': 'Охотники-любители отправляются в лес в начале сезона охоты. Они делятся друг с другом своими вымышленными и реальными подвигами, развлекаются и наслаждаются девственной природой. Неожиданно отдых героев омрачается необходимостью вступить в опасную схватку с браконьерами. Фильм «Ни пуха, ни пера!» повествует о красоте лесов и озёр, общем человеческом долге любить и хранить природу, её первозданность. Охотничья комедия Виктора Иванова показывает напряжённую и, в конечном счёте, успешную охоту настоящих охотников за браконьерами, полную невероятных приключений и неожиданных поворотов действия. Как и в каждой хорошей комедии, здесь тоже присутствуют песни и любовная история.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'бриллиантовая рука',
'text': '«Бриллиа́нтовая рука́» — советская эксцентрическая комедия, снятая в 1968 году режиссёром Леонидом Гайдаем. Сценарий написан Морисом Слободским, Яковом Костюковским и Леонидом Гайдаем. В основе картины — история о том, как зарубежные контрабандисты ошибочно приняли советского туриста Семёна Семёновича Горбункова за агента-перевозчика Геннадия Козодоева[⇨]. Сюжет раскручивается вокруг настойчивых, но безуспешных попыток злоумышленников вернуть себе бриллианты, спрятанные в гипсовой повязке[⇨]. Образ главного героя создавался сценаристами с учётом актёрских данных Юрия Никулина, который попал в съёмочную группу без кинопроб. В фильме также участвовали Андрей Миронов, Анатолий Папанов, Нонна Мордюкова, Светлана Светличная, Станислав Чекан, Владимир Гуляев, Нина Гребешкова и другие артисты[⇨]. Съёмки проходили с апреля по ноябрь 1968 года в Сочи, Адлере, Туапсе, Гжатске, Павловской Слободе, на натурных объектах Москвы и в павильонах «Мосфильма». «Зарубежные сцены» снимались в Баку[⇨]. При работе над отдельными сценами создатели фильма использовали комбинированные съёмки, механические спецэффекты и другие технологии[⇨].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)


i += 1
document = {
'title': 'Иван Васильевич меняет профессию',
'text': '«Ива́н Васи́льевич меня́ет профе́ссию» — советская фантастическая комедия 1973 года, снятая режиссёром Леонидом Гайдаем по мотивам пьесы Михаила Булгакова «Иван Васильевич». Фильм рассказывает об инженере-изобретателе Шурике, создавшем машину времени, которая открывает двери в XVI век — во времена Ивана Васильевича Грозного, в результате чего царь случайно оказывается в советской Москве, а его тёзка — управдом Иван Васильевич Бунша, вместе с вором-рецидивистом Жоржем Милославским — в палатах царя.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Кавказская пленница, или Новые приключения Шурика',
'text': '«Кавка́зская пле́нница, или Но́вые приключе́ния Шу́рика» — советский художественный фильм, снятый Леонидом Гайдаем. Сценарий к картине написан Яковом Костюковским и Морисом Слободским при участии самого режиссёра[⇨]. Съёмки проходили в 1966 году в павильонах «Мосфильма», а также в Крыму — на базе Ялтинской киностудии, на Кавказе — в районе Красной Поляны и в Абхазии — в долине озера Рица[⇨]. Главные роли исполняли Александр Демьяненко, Наталья Варлей, Владимир Этуш, Фрунзик Мкртчян, Руслан Ахметов, Юрий Никулин, Георгий Вицин, Евгений Моргунов и другие[⇨]. Действие фильма происходит на Кавказе. В основе сюжета — приключения собирателя фольклора Шурика, который, находясь в южном городе, оказывается втянутым в историю с похищением девушки Нины[⇨]. Инициатор похищения — номенклатурный работник районного масштаба товарищ Саахов; в роли его кунаков выступают персонажи комедийной троицы — Трус, Балбес и Бывалый[⇨].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Москва слезам не верит',
'text': 'Москва, 1958 год. В столичном общежитии живут три 20-летние подруги, приехавшие из провинции. Их судьбы складываются именно так, как предполагает характер каждой из девушек. Целеустремлённая и сдержанная Катерина, не добрав баллов при поступлении в институт, работает штамповщицей, а позже — слесарем-наладчиком на машиностроительном заводе. Стеснительная и простая Антонина трудится маляром на стройке и замуж выходит за москвича-рабочего. Самоуверенной и весёлой Людмиле Москва представляется лотереей, в которой она должна выиграть своё особенное счастье. Она работает на хлебозаводе, но мечтает выйти замуж за статусного москвича — артиста, управленца, спортсмена, учёного. Она знакомится с Сергеем Гуриным — восходящей хоккейной звездой. Катерина, которая пошла на поводу у Людмилы и вместе с ней выдала себя за профессорскую дочь, горько расплачивается за свою авантюру. Она полюбила красавца-телеоператора Рудольфа, забеременела от него, а он, узнав, где она на самом деле работает и что она обманула его, бросает её.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Служебный роман',
'text': 'Анатолий Ефремович мечтает о повышении — освободилось место начальника отдела лёгкой промышленности. И вот в один прекрасный день в статистическое учреждение заместителем директора назначают недавно вернувшегося из-за границы Юрия Григорьевича Самохвалова (Олег Басилашвили) — давнего институтского приятеля Новосельцева и бывшего возлюбленного Ольги Рыжовой. Самохвалов достаточно обеспечен, удачно женат, живёт с шиком. Ему, разумеется, понадобятся «свои люди» на новом месте, поэтому Самохвалов пытается намекнуть Людмиле Прокофьевне на Новосельцева как на приемлемый вариант для назначения на вакантное место, но эта кандидатура не находит поддержки. Тогда Юрий Григорьевич советует своему другу приударить за Калугиной вечером у него дома, где планируется отпраздновать назначение. Новосельцев, который смертельно боится Людмилы Прокофьевны, всё равно соглашается с предложением Самохвалова. Уединившись с ней, Анатолий Ефремович пытается угостить Калугину коктейлем, завести разговор о грибах, но Людмилу Прокофьевну ни он, ни предметы разговора не интересуют. Новосельцев уходит, но потом возвращается в несколько подвыпившем состоянии и безуспешно пытается развлечь Людмилу Прокофьевну стихами, песнями и танцами. Попытки угодить надоедают Людмиле Прокофьевне, и она выгоняет Новосельцева, а тот в ответ высказывает своему директору в присутствии коллег всё, что о ней думает. После этого Людмила Прокофьевна уезжает домой.'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Горе от ума',
'text': 'Замысел комедии возник в 1820 году (по некоторым данным, уже в 1816[2]), но активную работу над текстом автор начинает в Тифлисе по возвращении из Персии. Большое влияние на Грибоедова произвели рассказы его подруги детства Евгении Борисовны Греховой. К началу 1822 года написаны первые два акта, а весной и летом 1823 года в Москве завершается первый вариант пьесы. Именно здесь писатель мог пополнить наблюдения за бытом и нравами московского дворянства, «надышаться воздухом» светских гостиных. Но и потом работа не прекращается: в 1824 году возникает новый вариант, носивший название «Горе и нет ума» (первоначально — «Горе уму»).'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)

i += 1
document = {
'title': 'Евгений Онегин',
'text': '«Евге́ний Оне́гин» (дореф. «Евгеній Онѣгинъ») — роман в стихах русского поэта Александра Сергеевича Пушкина, начат 9 мая 1823 года и закончен 5 октября 1831 года, одно из самых значительных произведений русской словесности. Повествование ведётся от имени безымянного автора, который, впрочем, в первых же строфах называет Онегина «добрый мой приятель». По известному определению В. Г. Белинского, Пушкин назвал «Евгения Онегина» романом в стихах, поскольку в нём изображена «жизнь во всей её прозаической действительности»[1].'
}

response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

print('\nAdding document:')
print(response)



Adding document:
{'_index': 'python-test-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '2', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '3', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '4', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '5', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successfu

In [156]:
q = 'сильно бить'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)




q = 'астеник'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'очень далеко'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'неряшливость в одежде'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'все безразлично'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'ничего не сделано'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'здоровый человек'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'бесцельно'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'суеверие'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'только без рук'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'лепота'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'свадьбы не будет'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'а мы не хуже ихних'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'в женщине должна быть загадка'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)

q = 'ба знакомые все лица'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


q = 'чем меньше женщину мы любим тем легче нравимся мы ей'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'text']
    }
  },  "size": 3,
  "highlight": {
    "pre_tags": [
      "<strong>"
    ],
    "post_tags": [
      "</strong>"
    ],
    "fields": {
      "title": {},
      "text": {}
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results: ', q)
pp.pprint(response)
print ('######'*20)


Search results:  сильно бить
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'python-test-index',
                    '_score': 7.28474,
                    '_source': {'text': 'Драть как сидорову козу (прост.) – '
                                        'сильно, жестоко и безжалостно пороть, '
                                        'бить кого-либо. По наиболее '
                                        'распространенной версии, выражение '
                                        'связано с тем, что именем Сидор в '
                                        'народе называли зажиточного, но '
                                        'скупого и мелочного человеке. Далее '
                                        'объяснения оборота разнятся: 1) Сидор '
                                        'и свою козу даже за мелкую '
                                        'провинность драл (бил) безжалостно; '
         